# STIR reconstruction using data simulated with SIMIND ###
This notebook lays out a simple simulation and reconstruction using SIMIND and STIR \
Please see Rebecca Gillen's instructions / presentation for a more in depth guide

### simind can be donwloaded from https://simind.blogg.lu.se/downloads/

There are reasonably straight forward instructions to be followed for Windows/Mac/Linux
### STIR can be downloaded from https://github.com/UCL/STIR
for the current development version that has the SPECT projection matrix exposed in python or 
### Latest stable release: https://stir.sourceforge.net/

To use jupyter notebooks with this file, make sure you specify your stir python path
e.g for Linux:

    ~$PYTHONPATH /usr/local/python jupyter notebook

In [ ]:
### imports ###

# STIR modules
import stir
import stirextra

# Other modules
import numpy as np # STIR images & projection data can be exported as numpy arrays
import os
import matplotlib.pyplot as plt # for plotting


In [ ]:
# a useful function definition

def plot_show_save(image, fname, slice = False, save = False):
        ''' simple image plot and save function '''
        plt.subplot(111)
        if slice:
            plt.imshow(np.squeeze(stirextra.to_numpy(image))[slice])
        else:
            plt.imshow(np.squeeze(stirextra.to_numpy(image)))            
        plt.title(fname)
        plt.show()   
        if save:
            plt.savefig(fname)
            image.write_to_file('tmp_'+fname)


We are now ready to create our images. We'll use .par files in order to do this. SIMIND has some issues with switches (you'll see these later) getting confused with Linux directories so everything is currently in the same directory as the notebook.

The shell script creates images with .hv header files for STIR images, .smi header files for simind source images and .dmi header files for simind density maps.

We're currently having some issues with the simind attenuation, which we hope to fix soon so we'll only use the image in this simulation

In [ ]:
%%bash
sh generate_input_data.sh

We'll now load this image file into a STIR object

In [ ]:
image = stir.FloatVoxelsOnCartesianGrid.read_from_file("emission_image.hv") # example emission image 

And we have a simple image phantom

In [ ]:
slice = image.get_lengths()[1]//2 # middle slice
plot_show_save(image, "ground_truth", slice = slice)

We're now ready to simulate our emission data. We have a .smc file containing information about the simulation. Please read the simind manual to learn about the many different options available. These options can be altered using either the change command (type "change input.smc into the terminal) or using switches \
The syntax for reconstruction is as follows:
`simind input_file_prefix outpute_files_prefix`
This can be followed by switches seperated by forward slashes such as below \*\
\* Unfortunately this causes some trible with Linux & MacOS file directories. The SIMIND manual claims that two backslashed '\\' can be used in place of a forward slash that is part of a file directory, but I haven't found this to be the case

The following bash command defines a .smc file `input.smc` follwed by a prefix for output files `output` \
Switches are then used to define:
* /NN: a multiplier for the number of histories per projection (which is calculated using the sum of all voxel values)
* /PX: defines the image pixel size in the i,j direction (transverse in this case) - im.voxel_sizes()
* /FS: defines the prefix for the .smi emission image file
* /FD: defines the prefix for rhe .dmi attenuation image file # note: we've simulated this without attenuation

In [ ]:
%%bash
simind input output/NN:1/PX:0.4/FS:emission_image.smi

And (assuming the preious cell ran) we have now simulated our SPECT data!\
Next we need to get this data into a format the SIRF will recognise. Luckily we have a script ready that does this for us.
This script changes a few lines in the data's header file and the header file suffix. Differences between the conventions of interfiles in SIMIND and STIR/SIRF can be found in Rebecca's notes.

SIMIND (with the current scoring routine) will output an air, scatter and total sinogram. We're interested in the total

In [ ]:
%%bash
sh ./convertSIMINDToSTIR.sh output_tot_w1.h00

We can now view the resulting sinogram

In [ ]:
simind_projdata =  stir.ProjData.read_from_file("output_tot_w1.hs")

slice = simind_projdata.get_num_sinograms()//2
plot_show_save(simind_projdata, "simind_projdata", slice)

OK, so now we have our projection data in a format that SIRF likes, we can go about reconstructing the data.

In order to do this we first need to create our acqusition model matrix

In [ ]:
acq_model_matrix = stir.ProjMatrixByBinSPECTUB() # create a SPECT porjection matrix object
acq_model_matrix.set_keep_all_views_in_cache(False) # This keeps views in memory for a speed improvement
acq_model_matrix.set_resolution_model(0.1,0.1) # Set a resolution model (just a guess!)
acq_model_matrix.set_up(simind_projdata.get_proj_data_info(), image)

In [ ]:
projector = stir.ProjectorByBinPairUsingProjMatrixByBin(acq_model_matrix)
projector.set_up(simind_projdata.get_proj_data_info(), image)

In [ ]:
stir_projdata = stir.ProjDataInMemory(simind_projdata.get_exam_info(),
                                simind_projdata.get_proj_data_info())

projector.get_forward_projector().forward_project(stir_projdata, image)

slice = simind_projdata.get_num_sinograms()//2
plot_show_save(stir_projdata, "simind_projdata", slice)

And we can now backproject the data to get a rough idea of how we've done

In [ ]:
# create a dummy image to fill with out reconstructed image
target = image.get_empty_copy()
target.fill(1)

projector.get_back_projector().back_project(target, simind_projdata)
slice = target.get_lengths()[1]//2 # middle slice
plot_show_save(target, "backprojected_image", slice = slice)

We'll now use OSEM to reconstruct a nicer image.
First, we'll set up an objective function using out simind data and the projector we've just made from

Then we'll set up a reconstructor object with this objective function.

In [ ]:
# create our objective function
obj_function = stir.PoissonLogLikelihoodWithLinearModelForMeanAndProjData3DFloat()
obj_function.set_proj_data_sptr(simind_projdata)
obj_function.set_projector_pair_sptr(projector)

# and now our reconstruction object
recon = stir.OSMAPOSLReconstruction3DFloat()
recon.set_objective_function(obj_function)
recon.set_num_subsets(9) # This needs to be a divisor of the number of projections (72)
recon.set_num_subiterations(9) # we'll go through the subiterations on ce
recon.set_max_num_full_iterations(1)

In [ ]:
# create a dummy image to fill with out reconstructed image
target = image.get_empty_copy()
target.fill(1)

# and reconstruct
recon.set_up(target)
s = recon.reconstruct(target)

In [ ]:
slice = stir_projdata.get_num_sinograms()//2
# The recpnstructed image
plot_show_save((target), "reconstructed_image", slice = slice)

In [ ]:
### delete any rogue projections from the reconstruction
import glob
import os

for f in glob.glob("tmp*"):
    os.remove(f)